재표집법 (앙상블, cross-validation), 기계학습, 허태영

---



# 01.Classification

In [ ]:
# !pip install rpy2

In [ ]:
%load_ext rpy2.ipython

## 1.데이터 불러오기 & 데이터 전처리

### R 코드

In [ ]:
%%R

install.packages("caret")

library(caret)
mydata <- read.csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

print(head(mydata))
print(str(mydata))

### 변수 factor화
mydata$admit <- as.factor(mydata$admit)
mydata$rank <- as.factor(mydata$rank)

print(summary(mydata))

#### training & test data 분할 
set.seed(123)
my.ind      <- createDataPartition(y=mydata$admit, p=0.75, list=FALSE)
my.training <- mydata[my.ind, ]
my.test     <- mydata[-my.ind, ] # cross-validation에서 validation step을 진행하기 때문에 결과비교를 위해 test set를 따로 분할


### Python 코드

In [ ]:
import pandas as pd

data = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

#data 형태 탐색
print(data.head())
print(data.info())

#datatype 변환
data["admit"] = data["admit"].astype('category')
data["rank"] = data["rank"].astype('category')

# 기술통계량 확인
print(data.describe())

# 독립변수, 종속변수 나누기
y = data.pop("admit")
X = data

# test, train data split 
import  random
from sklearn.model_selection import train_test_split

random.seed(123)
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size=0.25)

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    int64  
 1   gre     400 non-null    int64  
 2   gpa     400 non-null    float64
 3   rank    400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB
None
              gre         gpa
count  400.000000  400.000000
mean   587.700000    3.389900
std    115.516536    0.380567
min    220.000000    2.260000
25%    520.000000    3.130000
50%    580.000000    3.395000
75%    660.000000    3.670000
max    800.000000    4.000000


## 2.모델 적합 및 cross-validation

### R코드

In [ ]:
%%R

cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.bag  <- train(admit ~ .,
                data       = my.training,
                trControl  = cctrl1,
                method     = "bag",
                tuneGrid   = data.frame(vars = seq(5, 20, by = 5)),
                bagControl = bagControl(fit = ldaBag$fit,
                                        predict = ldaBag$pred,
                                        aggregate = ldaBag$aggregate)
)

In [ ]:
%%R

cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.gbm  <- train(admit ~ .,
                data       = my.training,
                trControl  = cctrl1,
                method     = "gbm",
                tuneLength = 3
)


In [ ]:
%%R

cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.ada  <- train(admit ~ .,
                data       = my.training,
                trControl  = cctrl1,
                method     = "ada",
                tuneLength = 3
)

In [ ]:
%%R

cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.rf  <- train(admit ~ .,
                data       = my.training,
                trControl  = cctrl1,
                method     = "rf",
                tuneLength = 3
)


1 package is needed and is not installed. (randomForest). Would you like to try to install it now?
1: yes
2: no

Selection: 1


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/randomForest_4.7-1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 80852 bytes (78 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[w

### Python 코드

In [ ]:
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import cross_validate,GridSearchCV
import multiprocessing

#### bagging

In [ ]:
cross_validate(
    estimator=BaggingClassifier(),
    X=X, y=y,
    cv=5,
    n_jobs = multiprocessing.cpu_count(),
    verbose=True
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.7s finished


{'fit_time': array([0.03287721, 0.03323817, 0.02923322, 0.03241754, 0.03102231]),
 'score_time': array([0.00568914, 0.00586152, 0.00516915, 0.00588632, 0.00571156]),
 'test_score': array([0.7   , 0.725 , 0.7   , 0.6   , 0.5875])}

In [ ]:
bagging_param_grid = [{'n_estimators' : [10,20],
               'max_features' : [1,2] }]

In [ ]:
bagging_gs = GridSearchCV(
    estimator  = BaggingClassifier(),
    param_grid=bagging_param_grid,
    n_jobs = multiprocessing.cpu_count(),
    verbose=True
)

In [ ]:
bagging_gs.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=BaggingClassifier(), n_jobs=2,
             param_grid=[{'max_features': [1, 2], 'n_estimators': [10, 20]}],
             verbose=True)

In [ ]:
bagging_gs.best_estimator_

BaggingClassifier(max_features=2)

In [ ]:
model_bagging = bagging_gs.best_estimator_
model_bagging.fit(X_train,y_train)

BaggingClassifier(max_features=2)

#### GradientBoosting

In [ ]:
cross_validate(
    estimator=GradientBoostingClassifier(),
    X=X, y=y,
    cv=5,
    n_jobs = multiprocessing.cpu_count(),
    verbose=True
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.3s finished


{'fit_time': array([0.12860942, 0.1252346 , 0.1131866 , 0.11145401, 0.070894  ]),
 'score_time': array([0.00443792, 0.01135421, 0.00411296, 0.00422931, 0.0027566 ]),
 'test_score': array([0.7375, 0.7375, 0.675 , 0.6   , 0.65  ])}

#### AdaBoost

In [ ]:
cross_validate(
    estimator=AdaBoostClassifier(),
    X=X, y=y,
    cv=5,
    n_jobs = multiprocessing.cpu_count(),
    verbose=True
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.4s finished


{'fit_time': array([0.11764789, 0.11872411, 0.10783529, 0.10820365, 0.0866735 ]),
 'score_time': array([0.01597404, 0.02168345, 0.0159204 , 0.01533651, 0.00861502]),
 'test_score': array([0.75  , 0.7625, 0.7125, 0.675 , 0.5625])}

#### RandomForest

In [ ]:
cross_validate(
    estimator=RandomForestClassifier(),
    X=X, y=y,
    cv=5,
    n_jobs = multiprocessing.cpu_count(),
    verbose=True
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.7s finished


{'fit_time': array([0.25957084, 0.28618908, 0.2363565 , 0.23178315, 0.14964986]),
 'score_time': array([0.02427626, 0.02831316, 0.02264071, 0.02156234, 0.0116353 ]),
 'test_score': array([0.675 , 0.775 , 0.625 , 0.625 , 0.5625])}

## 3.confusionMatrix

### R코드

In [ ]:
%%R

bag.pred   <- predict(model.bag, newdata = my.test)
gbm.pred   <- predict(model.gbm, newdata = my.test)
ada.pred   <- predict(model.ada, newdata = my.test)
rf.pred    <- predict(model.rf, newdata = my.test)

In [ ]:
%%R

confusionMatrix(my.test$admit, bag.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 64  4
         1 24  7
                                          
               Accuracy : 0.7172          
                 95% CI : (0.6178, 0.8031)
    No Information Rate : 0.8889          
    P-Value [Acc > NIR] : 0.9999993       
                                          
                  Kappa : 0.2025          
                                          
 Mcnemar's Test P-Value : 0.0003298       
                                          
            Sensitivity : 0.7273          
            Specificity : 0.6364          
         Pos Pred Value : 0.9412          
         Neg Pred Value : 0.2258          
             Prevalence : 0.8889          
         Detection Rate : 0.6465          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.6818          
                                          
       'Positive' Class : 0               
                                    

In [ ]:
%%R

confusionMatrix(my.test$admit, gbm.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 62  6
         1 27  4
                                          
               Accuracy : 0.6667          
                 95% CI : (0.5648, 0.7582)
    No Information Rate : 0.899           
    P-Value [Acc > NIR] : 1.0000000       
                                          
                  Kappa : 0.05            
                                          
 Mcnemar's Test P-Value : 0.0004985       
                                          
            Sensitivity : 0.6966          
            Specificity : 0.4000          
         Pos Pred Value : 0.9118          
         Neg Pred Value : 0.1290          
             Prevalence : 0.8990          
         Detection Rate : 0.6263          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.5483          
                                          
       'Positive' Class : 0               
                                    

In [ ]:
%%R

confusionMatrix(my.test$admit, ada.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 61  7
         1 26  5
                                          
               Accuracy : 0.6667          
                 95% CI : (0.5648, 0.7582)
    No Information Rate : 0.8788          
    P-Value [Acc > NIR] : 1.000000        
                                          
                  Kappa : 0.07            
                                          
 Mcnemar's Test P-Value : 0.001728        
                                          
            Sensitivity : 0.7011          
            Specificity : 0.4167          
         Pos Pred Value : 0.8971          
         Neg Pred Value : 0.1613          
             Prevalence : 0.8788          
         Detection Rate : 0.6162          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.5589          
                                          
       'Positive' Class : 0               
                                    

In [ ]:
%%R

confusionMatrix(my.test$admit, rf.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 63  5
         1 26  5
                                          
               Accuracy : 0.6869          
                 95% CI : (0.5859, 0.7764)
    No Information Rate : 0.899           
    P-Value [Acc > NIR] : 1.000000        
                                          
                  Kappa : 0.1076          
                                          
 Mcnemar's Test P-Value : 0.000328        
                                          
            Sensitivity : 0.7079          
            Specificity : 0.5000          
         Pos Pred Value : 0.9265          
         Neg Pred Value : 0.1613          
             Prevalence : 0.8990          
         Detection Rate : 0.6364          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.6039          
                                          
       'Positive' Class : 0               
                                    

### Python코드

In [ ]:
bagging_predict = model_bagging.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true=y_test, y_pred=bagging_predict)

array([[60, 18],
       [13,  9]])

# 02.Regression

In [ ]:
%%R

reg.data <- read.csv(
  "https://stats.idre.ucla.edu/wp-content/uploads/2019/02/elemapi2v2.csv")

regdata.ind   <- createDataPartition(y = reg.data$api00 , p = 0.75, list = FALSE)
reg.training  <- reg.data[regdata.ind, ]
reg.test      <- reg.data[-regdata.ind, ]

summary(reg.training)

###bagging & cross-validation

install.packages("party")

cctrl1 <- trainControl(method = "cv", number = 5)

model.bagr  <- train(api00 ~ enroll + full + col_grad + emer,
                data       = reg.training,
                trControl  = cctrl1,
                B = 100,
                method     = "bag",
                tuneGrid   = expand.grid(vars = c(10, 15, 20)),
                bagControl = bagControl(fit       = ctreeBag$fit,
                                        predict   = ctreeBag$pred,
                                        aggregate = ctreeBag$aggregate)
)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘TH.data’, ‘libcoin’, ‘matrixStats’, ‘multcomp’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘zoo’, ‘sandwich’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/TH.data_1.1-1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8612708 bytes (8.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [ ]:
%%R

### gradient boosting & cross-validation
cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.gbmr  <- train(api00 ~ enroll + full + col_grad + emer,
                data       = reg.training,
                trControl  = cctrl1,
                method     = "gbm",
                tuneLength = 3
)

### adaboost는 caret 패키지에서 Regression 불가
# cctrl1 <- trainControl(method = "cv", number = 5)
# set.seed(123)
# model.adar  <- train(api00 ~ enroll + full + col_grad + emer,
#                 data       = reg.training,
#                 trControl  = cctrl1,
#                 method     = "ada",
#                 tuneLength = 3
# )

### random forest & cross-validation
cctrl1 <- trainControl(method = "cv", number = 5)
set.seed(123)
model.rfr  <- train(api00 ~ enroll + full + col_grad + emer,
                data       = reg.training,
                trControl  = cctrl1,
                method     = "rf",
                tuneLength = 3
)



Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1    17952.2221            -nan     0.1000 1530.1486
     2    16766.7634            -nan     0.1000 1058.6007
     3    15791.6263            -nan     0.1000  878.3576
     4    14831.9977            -nan     0.1000  747.4643
     5    14097.2723            -nan     0.1000  550.0329
     6    13120.7030            -nan     0.1000  796.7323
     7    12295.4718            -nan     0.1000  535.1391
     8    11710.7094            -nan     0.1000  387.9118
     9    11274.3929            -nan     0.1000  331.8554
    10    10784.8527            -nan     0.1000  436.3488
    20     7989.4969            -nan     0.1000   87.1437
    40     6521.9116            -nan     0.1000   20.6636
    60     6123.3291            -nan     0.1000   -0.5880
    80     5882.3495            -nan     0.1000   -9.9146
   100     5705.8584            -nan     0.1000  -64.7815
   120     5584.7630            -nan     0.1000  -28.8093
   140     544

In [ ]:
%%R

### 예측 및 평가 측도 계산
bagr.pred  <- predict(model.bagr, newdata = reg.test)
gbmr.pred  <- predict(model.gbmr, newdata = reg.test)
# adar.pred  <- predict(model.adar, newdata = reg.test)
rfr.pred   <- predict(model.rfr, newdata = reg.test)

In [ ]:
%%R

install.packages("forecast")
library(forecast)

accuracy(ts(bagr.pred), reg.test$api00) 


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘tseries’, ‘urca’, ‘RcppArmadillo’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]

               ME   RMSE      MAE       MPE     MAPE
Test set -7.98548 91.675 73.16672 -3.750902 12.24636


In [ ]:
%%R

accuracy(ts(gbmr.pred), reg.test$api00) 

                ME     RMSE      MAE       MPE     MAPE
Test set -7.398235 89.03668 69.25515 -3.462084 11.77936


In [ ]:
%%R

# accuracy(ts(model.adar), my.test$admit) 

NULL


In [ ]:
%%R

accuracy(ts(rfr.pred), reg.test$api00) 

               ME     RMSE      MAE       MPE     MAPE
Test set -10.9107 85.53554 67.06743 -3.888094 11.42478
